In [0]:
!pip install PyDrive
!pip install rouge

     |████████████████████████████████| 993kB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import json, gzip
import torch
import torch.nn as nn
import numpy as np
import time
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.__version__)

cuda:0
1.1.0


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Data for training set = 10k dev set = 1k
download = drive.CreateFile({'id': '1yJB36Jj1jw3U0JwE3IwhBYDY3VI1oCGd'})
download.GetContentFile('train.p')
download = drive.CreateFile({'id': '1T6YxqByvjNlCSZrU7FVmdCYlxM0RHlgw'})
download.GetContentFile('dev.p')
download = drive.CreateFile({'id': '1Nja8rXISqKfTSEbIPKYaf6Ey_Ud_Fik7'})
download.GetContentFile('test.p')
download = drive.CreateFile({'id': '1vPOGkgumTOVQ4N84GBo8AAPeAgwfyMep'})
download.GetContentFile('index2word.p')
download = drive.CreateFile({'id': '1JRYv-RQqhX6-fdYVUmyZmP9tBOtZjp_w'})
download.GetContentFile('word2index.p')

In [0]:
# #Data for training set = 50k dev set = 5k
# download = drive.CreateFile({'id': '1xq4JlZoWZ8_eO5F6VNBQvk2Mvub1Etpb'})
# download.GetContentFile('trainL.p')
# download = drive.CreateFile({'id': '1G3YAtOtrfv3PswNSHyGm0SIZy60n95zU'})
# download.GetContentFile('devL.p')
# download = drive.CreateFile({'id': '1T7rj8WokLSEEbc7-nqH48mjSnwBt4hEq'})
# download.GetContentFile('testL.p')
# download = drive.CreateFile({'id': '1bZXADxMyEn-yWtneA1sxDouGpuce1LRh'})
# download.GetContentFile('index2wordL.p')
# download = drive.CreateFile({'id': '1IMnW7Oyq4q9Ho5z87MbmVwlODjzEvY9h'})
# download.GetContentFile('word2indexL.p')

In [0]:
train_dict = pickle.load( open( "train.p", "rb" ) )
dev_dict = pickle.load( open( "dev.p", "rb" ) )
test_dict = pickle.load( open( "test.p", "rb" ) )
index2word = pickle.load( open( "index2word.p", "rb" ) )
word2index = pickle.load( open( "word2index.p", "rb" ) )

In [0]:
train_dict = sorted(train_dict, key = lambda i: len(i['text']), reverse = True)
dev_dict = sorted(dev_dict, key = lambda i: len(i['text']), reverse = True)
test_dict = sorted(test_dict, key = lambda i: len(i['text']), reverse = True)

In [0]:
train_dict = train_dict[:10]
dev_dict = dev_dict[:10]
test_dict = test_dict[:10]

In [0]:
train_dict_text = []
for i, data in enumerate(train_dict):
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
#         if i==789:
#             print(s, summ)
    t['summary'] = summ
    train_dict_text.append(t)
print(len(train_dict_text))

dev_dict_text = []
for data in dev_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    dev_dict_text.append(t)
print(len(dev_dict_text))

test_dict_text = []
for data in test_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    test_dict_text.append(t)
print(len(test_dict_text))

10
10
10


In [0]:
ind = 0
print(train_dict_text[ind]['summary'])
print(train_dict[ind]['summary'])
print(index2word[train_dict[ind]['summary'][-2]])
print(test_dict_text[ind]['summary'])
print(test_dict[ind]['summary'])
print(index2word[test_dict[ind]['summary'][-2]])

bradford city and port vale play out a goalless draw on the opening day of the league one season .
[3228, 1932, 18, 3230, 3231, 3082, 199, 13, 3233, 3234, 134, 38, 847, 410, 46, 38, 1189, 405, 963, 50]
season
a blackout that cut power to all of south australia has prompted thousands to take to social media .
[13, 4863, 250, 6296, 3141, 21, 502, 46, 2863, 3383, 52, 13140, 1182, 21, 37, 21, 2143, 11, 50]
media


In [0]:
import nltk
import nltk.data
from nltk import tokenize
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pickle
import json, gzip
import time
import math
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import euclidean
nltk.download('punkt')

W0508 20:47:14.214939 139833398060928 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Instructions for updating:
Colocations handled automatically by placer.


W0508 20:47:27.937835 139833398060928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed = hub.Module(module_url)
    embedded_text = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

In [0]:
ext_train_data = []
ext_dev_data = []
ext_test_data = []

for i, data in enumerate(train_dict_text):
    if i%1000==0:
        print(i)
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    try:
        sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
        num_sent = sentences_embeddings.shape[0]
        k = round(math.sqrt(num_sent))
        clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
        cluster_centers = clustering.cluster_centers_
        center_dist = defaultdict(list)
        for ns in range(num_sent):
            label = clustering.labels_[ns]
            dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
            center_dist[label].append((dist, ns))

        final = []
        for labels in range(k):
            try:
                center_dist[labels].sort(key = lambda x: x[0])
                final.append(center_dist[labels][0][1])
            except:
                continue
        final.sort()
        ex = []
        for i in final:
            ex.append(sentences_list[i])
        extractive = (' ').join(ex)
        dict_ = {}
        dict_['text'] = extractive[:]
        dict_['summary'] = data['summary'][:]
        ext_train_data.append(dict_)
    except:
        print('error')
end = time.time()
print(end-start)

for i, data in enumerate(dev_dict_text):
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive[:]
    dict_['summary'] = data['summary'][:]
    ext_dev_data.append(dict_)
end = time.time()
print(end-start)

for i, data in enumerate(test_dict_text):
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive[:]
    dict_['summary'] = data['summary'][:]
    ext_test_data.append(dict_)
end = time.time()
print(end-start)

0
0.03281283378601074
0.033701419830322266
0.03316330909729004


In [0]:
ext_train_data = sorted(ext_train_data, key = lambda i: len(i['text']), reverse = True)
ext_dev_data = sorted(ext_dev_data, key = lambda i: len(i['text']), reverse = True)
ext_test_data = sorted(ext_test_data, key = lambda i: len(i['text']), reverse = True)

In [0]:
ind = 2
print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])
print(ext_dev_data[ind]['text'])
print(ext_dev_data[ind]['summary'])
print(ext_test_data[ind]['text'])
print(ext_test_data[ind]['summary'])

today the company has announced a partnership with silicon solutions maker marvell that will make that concept a reality by 2011 — and for just $100 per device . marvell co-founder weili dai also told the new york times that the first version of the tablet would be equipped with google’s android operating system . watch the video below for more information on the tablet .
one laptop per child has teamed up with silicon solutions maker marvell to create $100 tablets by 2011 .
that’s a bit harder , as instagram disabled twitter UNK back in UNK long after facebook bought instagram in a $1-billion deal . to get that bad boy on twitter as an embedded video , simply upload the  UNK to youtube (preferably over wi-fi , as big  UNK files can really eat into your data plan) . once your hyperlapse is up on youtube , simply tweet out the youtube link and your video will appear embedded in the tweet:
it 's a youtube-based workaround
researchers studied the UNK size of 144 UNK collected during abori

In [0]:
from nltk.tokenize import word_tokenize
ind2word = {0: 'PAD', 1:'BOS', 2:'EOS', 3:'UNK'}
word2ind = {'PAD':0, 'BOS':1, 'EOS':2, 'UNK':3}
count = 4
for data in ext_train_data:
    text = data['text'][:]
    summ = data['summary'][:]
    for words in word_tokenize(text):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1

    for words in word_tokenize(summ):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1

In [0]:
ind = 2
# print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])

one laptop per child has teamed up with silicon solutions maker marvell to create $100 tablets by 2011 .


In [0]:
train_dict = []
dev_dict = []
train_maxlen = 30
dev_maxlen = 30

for i, data in enumerate(ext_train_data):
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    
    for words in word_tokenize(summ):
        summ_ind.append(word2ind[words])
    summ_ind = [1]+summ_ind+[2]
    while len(summ_ind)<train_maxlen:
        summ_ind.append(0)
    if len(summ_ind)==train_maxlen:
        data_point['text'] = word_tokenize(text)
        data_point['text_len'] = len(data_point['text'])
        data_point['summary_ind'] = summ_ind[:]
        data_point['summary'] = (['BOS']+word_tokenize(summ)+['EOS'])[:]
        train_dict.append(data_point)

    
    
for i, data in enumerate(ext_dev_data):
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    
    for words in word_tokenize(summ):
        if words in word2ind:
            summ_ind.append(word2ind[words])
        else:
            summ_ind.append(word2ind['UNK'])
    summ_ind = [1]+summ_ind+[2]
    if len(summ_ind)<dev_maxlen:
        summ_ind += [0]*(dev_maxlen-len(summ_ind))
    
    if len(summ_ind)==dev_maxlen:
        data_point['text'] = word_tokenize(text)
        data_point['text_len'] = len(data_point['text'])
        data_point['summary_ind'] = summ_ind[:]
        data_point['summary'] = (['BOS']+word_tokenize(summ)+['EOS'])[:]
        dev_dict.append(data_point)

In [0]:
data = {}
data['train'] = train_dict[:]
data['dev'] = dev_dict[:]
data['test'] = test_dict[:]
s = (data['dev'][9]['summary_ind'])
print(s)

[1, 284, 134, 3, 3, 3, 178, 65, 8, 3, 7, 294, 3, 3, 3, 188, 8, 69, 3, 3, 3, 65, 2, 0, 0, 0, 0, 0, 0, 0]


In [0]:
!pip install allennlp
from allennlp.modules.elmo import Elmo
from allennlp.modules.elmo import batch_to_ids

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 1, dropout=0, requires_grad = True)

In [0]:
trunc_norm_init_std = 1e-4
def init_linear_wt(linear):
    linear.weight.data.normal_(std=trunc_norm_init_std)
    if linear.bias is not None:
        linear.bias.data.normal_(std=trunc_norm_init_std)

In [0]:
class biEncoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(biEncoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.emb_size = params['emb_size']
        self.vocab_size = params['vocab_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True, num_layers=self.num_layers, bidirectional = True)
        self.reduce_h = nn.Linear(self.hidden_size * 2, self.hidden_size)
        init_linear_wt(self.reduce_h)
        self.reduce_c = nn.Linear(self.hidden_size * 2, self.hidden_size)
        init_linear_wt(self.reduce_c)
        self.elmo = Elmo(options_file, weight_file, 1, requires_grad=True, dropout=0)

    def forward(self, input, hidden, memory, seq_lengths):
        batch = len(input)
        character_ids = batch_to_ids(input).cuda()
        emb = (self.elmo(character_ids)['elmo_representations'][0]).cuda()
        #emb = self.embedding(input)
        #packed_emb = pack_padded_sequence(emb, (seq_lengths).numpy(), batch_first = True, enforce_sorted=False)
        #packed_output, (h, c) = self.lstm(packed_emb, (hidden, memory))
        #output, _ = pad_packed_sequence(packed_output, batch_first = True)
        output, (h, c) = self.lstm(emb, (hidden, memory))
        output1 = output[ :, :, :self.hidden_size]
        output2 = output[:, :, self.hidden_size:] #reverse
        output = output1+output2
        h = torch.cat(list(h), dim=1)                                   
        c = torch.cat(list(c), dim=1)
        h_reduced = F.relu(self.reduce_h(h))                       
        c_reduced = F.relu(self.reduce_c(c))
        return output, h_reduced.unsqueeze(dim=0), c_reduced.unsqueeze(dim=0)

    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
class AttnDecoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.vocab_size = params['vocab_size']
        self.emb_size = params['emb_size']
        self.num_layers = params['num_layers']
        self.batch_size = params['batch_size']
        self.embedding = nn.Embedding(self.vocab_size, self.hidden_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first=True, num_layers = self.num_layers)
        self.w = nn.Linear(2*self.hidden_size, self.vocab_size)
        self.p_gen_linear = nn.Linear(self.hidden_size + self.emb_size, 1)
        self.elmo = Elmo(options_file, weight_file, 1, requires_grad=True, dropout=0)

    def forward(self, input, hidden, memory, encoder_outputs, pad_ind):
        pointer_gen = True
        batch_size = len(input)
        #embedded = self.embedding(input)
        character_ids = batch_to_ids(input).cuda()
        #embedded = self.embedding(input)
        #embedded = self.dropout(embedded)
        embedded = (self.elmo(character_ids)['elmo_representations'][0]).cuda()
        output,(h,c) = self.lstm(embedded, (hidden, memory))
        attn_values = torch.bmm(output,encoder_outputs.permute(0,2,1))
        dec_attn_values = torch.bmm(output,output.permute(0,2,1))
        for b in range(batch_size):
            attn_values[b,:,pad_ind[b]:] = -1e10
        mask = torch.ones_like(dec_attn_values).cuda()
        mask1 = -1e10*torch.ones_like(dec_attn_values).cuda()
        
        mask = torch.tril(mask, diagonal = -1)
        mask1 = torch.triu(mask1)
        mask = mask+mask1
        dec_attn_values = dec_attn_values*mask.float()
        
        total_attn_values = torch.cat((attn_values,dec_attn_values),2)
        total_outputs = torch.cat((encoder_outputs,output),1)
        
        sftmax = nn.Softmax(dim=2)
        attn_dist = sftmax(attn_values)# This is attention distribution
        
        total_attn_weights = sftmax(total_attn_values) 
        
        weight_attn = torch.bmm(total_attn_weights, total_outputs)
        final_hidden = torch.cat((output,weight_attn),2)
        scores = self.w(final_hidden)
        sftmax2 = nn.Softmax(dim=1)
        vocab_dist = sftmax2(scores)
        
        p_gen = None
        if pointer_gen:
            p_gen_input = torch.cat((embedded, output), 2)  # B x (hidden_dim + emb_dim)
            p_gen = self.p_gen_linear(p_gen_input)
            p_gen = torch.sigmoid(p_gen)
        vocab_dist_ = p_gen * vocab_dist
        attn_dist_ = (1 - p_gen) * attn_dist  
        return vocab_dist_, attn_dist_


    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
from rouge import Rouge

def calculate_rouge (hyps, refs):
    #print(hyps, refs)
    r = Rouge()
    scores = r.get_scores(hyps, refs, avg=True)
    return(scores)


In [0]:
def train_lm(dataset, params, net, fscore = False):
    
    #print(dataset[9920]['summary'])
    # since the first index corresponds to the PAD token, we just ignore it
    # when computing the loss
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    sftmax = nn.Softmax(dim=2)
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    num_examples = len(dataset)    
    batches = [(start, start + params['batch_size']) for start in range(0, num_examples, params['batch_size'])]
    
    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        #random.shuffle(batches)
        ref_summ = []
        gen_summ = []
        count = 0
        # for each batch, calculate loss and optimize model parameters            
        for b_idx, (start, end) in enumerate(batches):
            batch = dataset[start:end]
            
            batch_text = []
            batch_summary = []
            batch_pad_len = []
#             batch_summ_indexes = []
            batch_summ_indexes = np.zeros([len(batch), len(batch[0]['summary_ind'])])
            i = 0
            for d in batch:
                
                batch_text.append(d['text'])
                batch_summary.append(d['summary'])
                batch_pad_len.append(d['text_len'])
#                 batch_summ_indexes.append(d['summary_ind'])
                batch_summ_indexes[i] = d['summary_ind']
                #print(len(d['summary_ind']), len(d['summary']))
                
                i += 1
            
            
            batch_summ_indexes = torch.from_numpy(batch_summ_indexes).long().cuda()
#             batch_summ_indexes = torch.tensor(batch_summ_indexes).long.cuda()
            vocab_dist_, attn_dist_ = net(batch_text, batch_summary, batch_pad_len)
            _,_,l = attn_dist_.size()
            enc_batch_extend_vocab = batch_text.unsqueeze(1)
            enc_batch_extend_vocab = enc_batch_extend_vocab[:,:,:l]
            
            enc_batch_extend_vocab = enc_batch_extend_vocab.expand_as(attn_dist_)
            final_dist = vocab_dist_.scatter_add(2, enc_batch_extend_vocab, attn_dist_)
            
            targets = batch_summary[:, 1:].contiguous()
            final_dist = final_dist[:, :-1, :]
            gold_probs = torch.gather(final_dist, 2, targets.unsqueeze(2)).squeeze()
            nll = -torch.log(gold_probs +  1e-12)
            #remove padding loss
            for b in range(len(batch)):
                nll[b,batch_summ_len[b]-1:] = 0
            loss = torch.sum(nll)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss
            
            if fscore:
                logits = final_dist
                words = torch.argmax(logits, dim=2)
                for sent in range(words.size()[0]):
                    summ = ''
                    gold = ''
                    for word in range(words.size()[1]):
                        if ind2word[words[sent][word].item()] == 'EOS':
                            break
                        if summ :
                            summ += ' '
                        summ += ind2word[words[sent][word].item()]

                    for word in range(1,len(batch_summary[sent])-1):
                        if ind2word[batch_summary[sent][word].item()] == 'EOS':
                            break
                        if gold:
                            gold += ' '
                        gold += ind2word[batch_summary[sent][word].item()]

                    if summ == '.'*len(summ):
                        count += 1
                    if not summ or summ == '.'*len(summ):
                        summ = '====='

                    gen_summ.append(summ)
                    ref_summ.append(gold)
                    
        idx = 5
        logits = final_dist[idx,:,:].unsqueeze(0)
        words = torch.argmax(logits, dim=2).squeeze(0)
        gold_summ_last = ''
        summ_last = ''
        for l in range(batch_summary[idx,1:].size()[0]):
            gold_summ_last += ind2word[batch_summary[idx,l].item()] + ' '
        for l in range(words.size()[0]):
            summ_last += ind2word[words[l].item()] + ' '
        print('########################Train######################')
        print(gold_summ_last)
        print(summ_last)
        
        if fscore:
            rouge_score = calculate_rouge(gen_summ, ref_summ)
            print('rouge', rouge_score)
        
        loss, perplex =  compute_loss(data['dev'], net, params['batch_size'], fscore = True)
        print('epoch: %d, loss: %0.2f, time: %0.2f sec, dev loss: %0.2f, dev perplexity: %0.2f' %\
              (epoch, ep_loss, time.time()-start_time, loss, perplex))
        print('####################################################')



            pred, = net(batch_text, batch_summary, batch_pad_len)
            print((pred))
            sz = pred.size()[1]
            preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
            targets = batch_summ_indexes[:, 1:sz].contiguous().view(-1)
            #print(pred.size(),preds.size(),targets.size(),batch_summ_indexes.size())
            loss = criterion(preds, targets)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss
            
            if fscore:
                logits = sftmax(pred)
                words = torch.argmax(logits, dim=2)
                for sent in range(words.size()[0]):
                    summ = ''
                    gold = ''
                    for word in range(words.size()[1]):
                        if ind2word[words[sent][word].item()] == 'EOS':
                            break
                        if summ :
                            summ += ' '
                        summ += ind2word[words[sent][word].item()]

                    for word in range(1,len(batch_summary[sent])-1):
                        if ind2word[batch_summ_indexes[sent][word].item()] == 'EOS':
                            break
                        if gold:
                            gold += ' '
                        gold += ind2word[batch_summ_indexes[sent][word].item()]

                    if summ == '.'*len(summ):
                        count += 1
                    if not summ or summ == '.'*len(summ):
                        summ = '====='
                    if not gold or gold=='.'*len(gold):
                        continue

                    gen_summ.append(summ)
                    ref_summ.append(gold)
        print(start)   
        ind = 0
        scores = pred[ind,:,:].unsqueeze(0)
        logits = sftmax(scores)
        words = torch.argmax(logits, dim=2).squeeze(0)
        gold_summ_last = ''
        summ_last = ''
        for l in range(batch_summ_indexes[ind,1:].size()[0]):
            gold_summ_last += ind2word[batch_summ_indexes[ind,l].item()] + ' '
        for l in range(words.size()[0]):
            summ_last += ind2word[words[l].item()] + ' '
        print('########################Train######################')
        print(gold_summ_last)
        print(summ_last)
        
        #print(len(gen_summ), len(ref_summ))
        if fscore:
            rouge_score = calculate_rouge(gen_summ, ref_summ)
            print('rouge', rouge_score)
        
#         print('epoch: %d, loss: %0.2f, time: %0.2f sec' %\
#                 (epoch, ep_loss, time.time()-start_time))
        loss, perplex =  compute_loss(data['dev'], net, params['batch_size'], fscore = True)
        print('epoch: %d, loss: %0.2f, time: %0.2f sec, dev loss: %0.2f, dev perplexity: %0.2f' %\
              (epoch, ep_loss, time.time()-start_time, loss, perplex))
        print('####################################################')

In [0]:
def compute_loss(dataset, net, bsz=10, fscore = False):
    
    sftmax= nn.Softmax(dim=2)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
    batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    
    total_unmasked_tokens = 0. # count how many unpadded tokens there are
    nll = 0.
    gen_summ = []
    ref_summ = []
    count = 0
    for b_idx, (start, end) in enumerate(batches):
#         print(start, end)
        batch = dataset[start:end]
    
        batch_text = []
        batch_summary = []
        batch_pad_len = []
        batch_summ_indexes = np.zeros([len(batch), len(batch[0]['summary_ind'])])
        i = 0
        for d in batch:
            batch_text.append(d['text'])
            batch_summary.append(d['summary'])
            batch_pad_len.append(d['text_len'])
            batch_summ_indexes[i] = d['summary_ind']
            i += 1
                
        #batch_text = torch.from_numpy(np.array(batch_text)).long().cuda()
        #batch_summary = torch.from_numpy(np.array(batch_summary)).long().cuda()
        batch_pad_len = torch.from_numpy(np.array(batch_pad_len)).long().cuda()
        batch_summ_indexes = torch.from_numpy(batch_summ_indexes).long().cuda()
#        ut = torch.nonzero(batch_text).size(0)
        pred = net(batch_text, batch_summary, batch_pad_len)
        preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
        sz = pred.size()[1]
        targets = batch_summ_indexes[:, 1:sz].contiguous().view(-1)
        loss = criterion(preds, targets)
        nll += loss.detach()
 #       total_unmasked_tokens += ut
        
        if fscore:
            #print('pred', pred.size())
            logits = sftmax(pred)
            words = torch.argmax(logits, dim=2)
            #print('words', words.size())
            for sent in range(words.size()[0]):
                summ = ''
                gold = ''
                for word in range(words.size()[1]):
                    if ind2word[words[sent][word].item()] == 'EOS':
                        break
                    if summ :
                        summ += ' '
                    summ += ind2word[words[sent][word].item()]

                for word in range(1,len(batch_summ_indexes[sent])-1):
                    if ind2word[batch_summ_indexes[sent][word].item()] == 'EOS':
                        break
                    if gold:
                        gold += ' '
                    gold += ind2word[batch_summ_indexes[sent][word].item()]

                if summ == '.'*len(summ):
                    count += 1
                if not summ or summ == '.'*len(summ):
                    summ = '====='
                if not gold or gold=='.'*len(gold):
                    continue

                gen_summ.append(summ)
                ref_summ.append(gold)
        
    ind = 0
    scores = pred[ind,:,:].unsqueeze(0)
    logits = sftmax(scores)
    words = torch.argmax(logits, dim=2).squeeze(0)
    gold_summ = ''
    summ = ''
    for l in range(batch_summ_indexes[ind,1:].size()[0]):
        gold_summ += ind2word[batch_summ_indexes[ind,l].item()] + ' '
    for l in range(words.size()[0]):
        summ += ind2word[words[l].item()] + ' '
            
    
    print('#############Dev###########')
    if fscore:
        rouge_score = calculate_rouge(gen_summ, ref_summ)
        print('rouge', rouge_score)
    print(gold_summ)
    print(summ)
    
    perplexity = torch.exp(nll).cpu() #torch.exp(nll / total_unmasked_tokens).cpu()
    return nll, perplexity.data


In [0]:
# function to evaluate LM perplexity on some input data, DO NOT MODIFY
def test_lm(dataset, net, bsz=1, beam_search = False, max_len = 30):
    
    sftmax = nn.Softmax(dim=1)
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
    batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    nll = 0
    ans = []
    for b_idx, (start, end) in enumerate(batches):
            
        batch = dataset[start:end]
        batch_text = []
        batch_summary = []
        batch_pad_len = []
        batch_summ_indexes = np.zeros([len(batch), len(batch[0]['summary_ind'])])
        i = 0
        
        for d in batch:
                batch_text.append(d['text'])
                batch_summary.append(d['summary'])
                batch_pad_len.append(d['text_len'])
                batch_summ_indexes[i] = d['summary_ind']
                i += 1
                
#         batch_text = torch.from_numpy(batch_text).long().cuda()
#         batch_summary = torch.from_numpy(batch_summary).long().cuda()
#         batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        
#         batch_summary_gen = torch.ones([bsz, 1], dtype=torch.long).cuda()
#         bos = ['BOS']
        batch_summary_gen = [['BOS'] for i in range(bsz)]
        
        for i in range(max_len):    
            preds = net(batch_text, batch_summary_gen, batch_pad_len)[:,-1,:]
            distribution = sftmax(preds)
            ind = torch.argmax(distribution,1)
            for j in range(bsz):
              batch_summary_gen[j].append(index2word[ind[j].item()])
        
        ans+=batch_summary_gen.copy()
    
    gen_summ = []
    gold_summ = []
    
    for n in range(num_examples):
        summ = ''
#         summ = (' ').join(ans[n])
        for k in ans[n][1:]:
          if k == 'EOS':
            break
          summ += k + ' '  
        gen_summ.append(summ)
        
    for n in range(num_examples):
        summ = ''
#         summ = (' ').join(dataset[n]['summary'])
        for k in dataset[n]['summary'][1:]:
          if k == 'EOS':
            break
          summ += k + ' '
        gold_summ.append(summ)
    print((gold_summ))
    print((gen_summ))
    rouge_score = calculate_rouge(gen_summ, gold_summ)
    print('rouge', rouge_score)

In [0]:
class SuperAwesome(nn.Module):
    def __init__(self, params):
        super(SuperAwesome, self).__init__()
        self.bienc = biEncoderLSTM(params).cuda()
        self.attn_dc = AttnDecoderLSTM(params).cuda()
        self.vocab_size = params['vocab_size']
        #self.bienc.embedding.weight = self.attn_dc.embedding.weight
    def forward(self, batch_text, batch_summary, batch_pad_len):
        batch_size = len(batch_text)
        h_init,c_init = self.bienc.initHidden(batch_size)
        enc_out, h_enc, c_enc = self.bienc(batch_text,h_init,c_init, batch_pad_len)
        h_init, c_init = h_enc.contiguous(),c_enc.contiguous()
        vocab_dist_, attn_dist_ = self.attn_dc(batch_summary,h_init,c_init,enc_out, batch_pad_len)
        return vocab_dist_, attn_dist_

In [0]:
params = {}
params['hidden_size'] = 200
params['emb_size'] = 1024
params['num_layers'] = 1
params['batch_size'] = 50
params['vocab_size'] = len(word2ind)
params['learning_rate'] = 0.01
params['epochs'] = 30
net = SuperAwesome(params)

train_lm(data['train'], params, net, fscore = True)

In [0]:
test_len = []
#TEST TIME
gen_summ = []
ref_summ = []
for i in range(len(data['test'])):
    #print(i, end=',')
    y = data['test'][i]['summary']
    gold = ''
    for t in y:
        if(ind2word[t]=='SOS'):
            continue
        if(ind2word[t] == 'EOS'):
            break  
        gold += (ind2word[t])
        gold +=' '
    
    
    print("ground truth")  
    print(gold)
    net.eval()
    summ, l = test_lm(data['test'][i:i+1], net) 
    test_len.append(l)
    if summ == '.'*len(summ):
        count += 1
    if not summ or summ == '.'*len(summ):
        summ = '====='
    print(summ)
    gen_summ.append(summ)
    ref_summ.append(gold)
rouge_score = calculate_rouge(gen_summ, ref_summ)
print('Test rouge', rouge_score)  
# for n in range(num_examples):
#         summ = ''
#         for i in range(1,max_len+1):
#             if summ:
#                 summ += ' '
#             if int(ans[n,i]) == 2:
#                 break
#             summ += ind2word[int(ans[n,i])]
#         gen_summ.append(summ)